In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)

url = 'https://github.com/ananyadutta03/CVPR/blob/main/Mid/ClassTask/Book1.csv'

data = pd.read_csv(url)

print(data.head())
print(data.shape)

X = data[['X1', 'X2']].values
Y_labels = data['Label'].values

Y = np.zeros((Y_labels.shape[0], 3))
for i in range(Y_labels.shape[0]):
    Y[i, Y_labels[i]] = 1


In [ ]:
class NeuralNetwork(object):
    def __init__(self):
        inputLayerNeurons = 2
        hiddenLayer1Neurons = 10
        hiddenLayer2Neurons = 10
        outLayerNeurons = 3

        self.learning_rate = 0.2
        self.W_H1I = np.random.randn(inputLayerNeurons, hiddenLayer1Neurons)
        self.W_H2H1 = np.random.randn(hiddenLayer1Neurons, hiddenLayer2Neurons)
        self.W_OH2 = np.random.randn(hiddenLayer2Neurons, outLayerNeurons)

    def sigmoid(self, x, der=False):
        if der == True:
            return x * (1-x)
        else:
            return 1 / (1 + np.exp(-x))

    def feedForward(self, X):
        hidden1_input = np.dot(X, self.W_H1I)
        self.hidden1_output = self.sigmoid(hidden1_input)

        hidden2_input = np.dot(self.hidden1_output, self.W_H2H1)
        self.hidden2_output = self.sigmoid(hidden2_input)

        output_input = np.dot(self.hidden2_output, self.W_OH2)
        pred = self.sigmoid(output_input)
        return pred

    def backPropagation(self, X, Y, pred):
        output_error = Y - pred
        output_delta = self.learning_rate * output_error * self.sigmoid(pred, der=True)

        hidden2_error = output_delta.dot(self.W_OH2.T)
        hidden2_delta = self.learning_rate * hidden2_error * self.sigmoid(self.hidden2_output, der=True)

        hidden1_error = hidden2_delta.dot(self.W_H2H1.T)
        hidden1_delta = self.learning_rate * hidden1_error * self.sigmoid(self.hidden1_output, der=True)

        self.W_H1I += X.T.dot(hidden1_delta)
        self.W_H2H1 += self.hidden1_output.T.dot(hidden2_delta)
        self.W_OH2 += self.hidden2_output.T.dot(output_delta)

    def train(self, X, Y):
        output = self.feedForward(X)
        self.backPropagation(X,Y,output)

NN = NeuralNetwork()

err = []
for i in range(10000):
    NN.train(X,Y)
    err.append(np.mean(np.square(Y - NN.feedForward(X))))

plt.plot(err)
plt.xlabel('Iterations')
plt.ylabel('Error')
plt.show()

# Test predictions
predictions = NN.feedForward(X)
predicted_classes = np.argmax(predictions, axis=1)
accuracy = np.mean(predicted_classes == Y_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print some sample predictions
print("\nSample predictions:")
for i in range(5):
    print(f"Input: {X[i]}, Predicted class: {predicted_classes[i]}, Actual class: {Y_labels[i]}")

# Visualize the data and predictions
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X[:, 0], X[:, 1], c=Y_labels, cmap='viridis', edgecolors='k')
plt.title('Original Data')
plt.xlabel('X1')
plt.ylabel('X2')

plt.subplot(1, 2, 2)
plt.scatter(X[:, 0], X[:, 1], c=predicted_classes, cmap='viridis', edgecolors='k')
plt.title('Predictions')
plt.xlabel('X1')
plt.ylabel('X2')

plt.tight_layout()
plt.show()